In [80]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import csv; import requests
import time
import pandas as pd
import numpy as np
import os
import psutil

In [85]:
class kfc_web_scraper:
    def __init__(self):
        self.pick_up_css = "#app > div.kfc_view_box > div > div.__start_ordering_container > div > ul > li:nth-child(1) > a"
        self.drop_down_css = "#delivery > div > form > div > div > span > span > i"
        self.store_select_css = "ul.el-scrollbar__view.el-select-dropdown__list"
        self.drop_list_css = ".el-scrollbar__view"
        self.address_confirm_css = "html body.overflow div#app.__ishome div.kfc_view_box div.kfc_dialog div.__start_ordering_container div.__ordering_content div.tab-content div#delivery.tab-pane.active div.__pick_delivery_tab button.kfc_btn.__brand.__address_confirm_button"
        self.search_button = "#delivery > div > button.kfc_btn.__brand.__address_confirm_button"
        self.img = ".__product_item_image > a > .__product_im_img > img"
        self.name = ".__product_item_desc > a > h2"
        self.price = ".__product_item_desc > .__product_desc_price"
        self.main_menu = ".__main_menu_content"
        self.category = ".__main_content_section"
        self.items = ".__main_content_block > .__product_item:not([style])"
        self.url = "https://www.kfcmongolia.com/index"
        self.store_list_dict = ''
        self.main_Data_frame = pd.DataFrame(columns=["names", "prices", "categories", "image", "store"])
        
    def get_browser(self):
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('window-size=1000x1000')
        options.add_argument("disable-gpu")
        
        self.driver = webdriver.Chrome(
                                        os.getcwd() + '\chromedriver', 
                                        options=options,
#                                         service=Service(ChromeDriverManager().install())
        )
        self.driver.get(self.url)
        self.driver_process = psutil.Process(self.driver.service.process.pid)
        
        pick_up = self.driver.find_element(By.CSS_SELECTOR, self.pick_up_css)
        pick_up.click()
        
        drop_down = self.driver.find_element(By.CSS_SELECTOR, self.drop_down_css)
        drop_down.click()

    def get_store_dict(self):
        self.get_browser()
        
        time.sleep(2)
        stores = self.driver.find_element(By.CSS_SELECTOR, self.store_select_css).text
        store_list = stores.split('\n')
        self.store_list_dict = {i+1:s for i,s in enumerate(store_list)}
        
        self.quit_browser()

    def get_store_items(self):
        
        self.get_store_dict()
        
        if len(self.store_list_dict) > 0:
            for i in range(len(self.store_list_dict)):
                item_imgs = []
                item_names = []
                item_prices = []
                item_categories = []
                item_store = []
                
                self.get_browser()
                while True:
                    try:
                        input_store_names = self.driver.find_elements(By.CSS_SELECTOR, 'li.el-select-dropdown__item')
                        input_store_names[i].click()
                        if input_store_names[i].text == "":
                            continue 
                        break
                    except:
                        continue

                print(len(input_store_names), i, input_store_names[i].text)
                
                store_name = input_store_names[i].text
                search_button = self.driver.find_element(By.CSS_SELECTOR, self.search_button)
                search_button.click()

                time.sleep(2)
                get_main_menu = self.driver.find_element(By.CSS_SELECTOR, self.main_menu)
                WebDriverWait(self.driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, self.category)))
                get_categories = get_main_menu.find_elements(By.CSS_SELECTOR, self.category)
                

                for j in range(len(get_categories)):
                    while True:
                        try:
                            get_items = get_categories[j].find_elements(By.CSS_SELECTOR, self.items)
                            category = get_categories[j].find_element(By.CSS_SELECTOR, 'h3').text
                            break
                        except NoSuchElementException:
                            continue
                        except StaleElementReferenceException:
                            continue
                    
                    for k in range(len(get_items)):
                        img = get_items[k].find_element(By.CSS_SELECTOR, self.img).get_attribute("src")
                        name = get_items[k].find_element(By.CSS_SELECTOR, self.name).text
                        price = get_items[k].find_element(By.CSS_SELECTOR, self.price).text

                        if name != '':
                            item_imgs.append(img)
                            item_categories.append(category)
                            item_names.append(name)
                            item_prices.append(price)
                            item_store.append(store_name)

                self.quit_browser()
                    
                temp_DataFrame = pd.DataFrame({"names": item_names, 
                                               "prices": item_prices,
                                               "categories": item_categories,
                                               "image": item_imgs, 
                                               "store": item_store
                                              })
                self.main_Data_frame = pd.concat([self.main_Data_frame, temp_DataFrame])
            
        return self.main_Data_frame
            
    def quit_browser(self, ):
        self.driver.quit()

In [86]:
test = kfc_web_scraper()
df = test.get_store_items()

16 0 KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
16 1 KFC-Смарт /Өргөө кино театрын эсрэг талд/
16 2 KFC-Сөүл (Хүүхдийн 100)
16 3 KFC-Тэнгис /Тэнгис кино театрын хажууд/
16 4 KFC-Намянжу/(ХӨСҮТ дэргэд)
16 5 KFC-Энхтайван (24 цагийн салбар)
16 6 KFC-Хархорин /Хархорин захын буудал/
16 7 KFC-Зайсан салбар
16 8 KFC-Сансар /24 Цагийн салбар/
16 9 KFC-Морьтон салбар /19-н Буудал/
16 10 KFC-Хоум Плаза /Хан-Уул/ - 24 цагийн салбар
16 11 KFC-ГАЛЛЕРИА /Галлериа моол 1 давхарт/
16 12 KFC-Яармаг /Яармаг Номингийн 1 давхарт/
16 13 KFC-32 тойрог салбар
16 14 KFC-21 Салбар
16 15 KFC-Баянхошуу салбар


In [87]:
test_1 = list(df['store'].unique())

In [88]:
test_2 = test.store_list_dict

In [89]:
test_3 = [i for i in test_2.values()]

In [98]:
def count(arr, n):
    mp = dict()
    non_dup = []
    
    for i in range(n):
        if arr[i] in mp.keys():
            mp[arr[i]] += 1
        else:
            mp[arr[i]] = 1
    for x in mp:
        if mp[x] == 1:
            non_dup.append(x)
            
    if len(non_dup) > 0:
        print(non_dup)
    else:
        print('no non duplicates')

In [99]:
arr = test_1 + test_3
count(arr, len(arr))

no non duplicates


In [100]:
df

,names,prices,categories,image,store
0,10ш НАГГЕТ,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/upY59gK.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
1,20ш НАГГЕТ,"13,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/mMZXiqy.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
2,ТВИСТЕР,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/YVsecYB.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
3,ЗИНГЕР БУРГЕР,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/Fpqsu4Q.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
4,ФИЛЛЕ БУРГЕР,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/rOrATAA.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
...,...,...,...,...,...
55,MOUNTAIN DEW R,"1,000₮",УНДАА,https://imgur.com/bknkfQA.png,KFC-Баянхошуу салбар
56,ЦАЙ,"1,500₮",УНДАА,https://imgur.com/33LiW7o.png,KFC-Баянхошуу салбар
57,ЦЭВЭР УС,"1,000₮",УНДАА,https://imgur.com/1iiYLdP.png,KFC-Баянхошуу салбар
58,PEPSI L,"1,500₮",УНДАА,https://imgur.com/G7foBq9.png,KFC-Баянхошуу салбар
